# Stationary & Lags

## Stationary
Para poder modelar una serie temporal necesitamos que sea estacionaria, lo que significa que sus estadísticos no cambien con el tiempo. Estadísticos como la media, varianza y covarianza.

Veamos en un ejemplo de cómo comprobar si la serie es estacionaria y en caso contrario qué tenemos que hacer para convertirla.

Cargamos datos:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

series = pd.read_csv('data/FB.csv', header=0, index_col=0)
series.head()

Representamos la serie

Vemos que no es estacionaria. Presenta una tendencia y no es lo mismo la varianza en cada zona de la serie temporal

Calculamos sus estadísticos

Ejecutamos el test de Dickey-Fuller para comprobar si la serie temporal es estacionaria.

El test de DF plantea la hipótesis nula de que la serie NO es estacionaria. Se trata de comprobar sus p-values, si quedan por encima o por debajo su nivel de significación (0.05). Si queda por encima de su nivel de significación, aceptamos la hipótesis nula y concluimos que la serie NO es estacionaria. En caso contrario, rechazaríamos la hipótesis nula, en favor de la hipótesis alternativa, con la que concluiremos que la serie es estacionaria.

Para solucionar esto, diferenciamos la serie. Normalmente con una diferenciación de 1 es suficiente.

Representamos la diferenciación de 1

Ahora calculamos sus estadísticos

Ya que hemos diferenciado, y supuestamente ahora la serie es estacionaria, vamos a volver a probar con el test de Dickey-Fuller

#### Ejemplo AVG TEM ANUAL
Ahora utilizaremos los datos 'Mean_Temp_IMD_2017.csv'

In [ ]:
series_tmp = pd.read_csv('data/Mean_Temp_IMD_2017.csv', header=0, index_col=0)
series_tmp.head()

## Lag
Un lag es un retraso en el tiempo. Se utiliza este concepto para ver la relación que tiene cada instante de un time series con instantes anteriores.

Se pueden calcular mediante shift

Como los algoritmos de regresión en time series dependen de instantes anteriores, hay que intentar evitar que los lags estén muy autocorrelados, por que sino sufriremos de multicolinearidad.

Vamos a comprobar cuánto de correlados están los lags

## Autocorrelation plot
Tengo que ver cómo es la gráfica de autocorrelación, ya que si voy a utilizar una regresión lineal, no queremos que los lags estén muy correlacionados, unos con otros.

¿Cómo interpreto esto? Correlación muy alta entre cada dato y sus lags. Esto es debido a que la serie no es estacionaria.

Diferenciando, perdemos esa correlación entre los lags, y conseguimos una serie temporal más adecuada para entrenar nuestros modelos.

Probamos la autocorrelacion con los datos de bolsa y sus lags, sin diferenciar

Diferenciando, me baja mucho la correlación, que es lo que estamos buscando.